<a href="https://colab.research.google.com/github/srinithish/Deep-Learning/blob/master/HW2_Prob2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np
import sklearn
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import random
import matplotlib.cm as cm
import math

# Problem 2

Have used HE initialisation and AdamOptimizer.
The network structure, <br>.


First Conv Layer :  filter size with 4x4 and 16 filters <br>
Max Pool 2x2 <br>
Second Conv Layer : filter size with 2x2 and 32 filters <br>
Max Poll 2x2  <br>
and Fully Connected layers of dimensions as : [32x129x5,800,513] <br>
Dropout at the final layer


In [0]:
###load files
# !pip install librosa # in colab, you'll need to install this
import librosa
import io


"""
Change directory path here and leave it blank if its in the same directory as the 
Notebook
"""
# dirpath  = "./drive/DeepLearning Assign1/"
dirpath  = ""


### Loading the files

##Y path  = 
sn_train_y, sr=librosa.load(dirpath+'train_clean_male.wav', sr=None)
S_Train=librosa.stft(sn_train_y, n_fft=1024, hop_length=512)

##X
sn_train_x, sr=librosa.load(dirpath+'train_dirty_male.wav', sr=None)
X_Train=librosa.stft(sn_train_x, n_fft=1024, hop_length=512)

##XTest1
sn_test1, sr=librosa.load(dirpath+'test_x_01.wav', sr=None)
X1_test =librosa.stft(sn_test1, n_fft=1024, hop_length=512)


##XTest2
sn_test2, sr=librosa.load(dirpath+'test_x_02.wav', sr=None)
X2_test =librosa.stft(sn_test2, n_fft=1024, hop_length=512)

X1_test = np.transpose(X1_test)
X1_Test_Mag = np.abs(X1_test)

X2_test = np.transpose(X2_test)
X2_Test_Mag = np.abs(X2_test)

S_Mag = np.transpose(np.abs(S_Train))
X_Mag = np.transpose(np.abs(X_Train))

# print(X1_test.shape)

In [0]:
def gen2DImageFromAudio(XMagFrequecyDom,windowSize):
  
  ##function to generate the 20 window images of the audio spectra
  maxIndex = XMagFrequecyDom.shape[0]
  
  
  listOfData = []
  for i in range(0,maxIndex-windowSize+1):
    listOfData.append(XMagFrequecyDom[i:i+windowSize,:])
  
  
 
  
  return np.array(listOfData)
  
  

  
gen2DImageFromAudio(X_Mag,20).shape
    

(2440, 20, 513)

In [0]:


##network params
learning_rate = tf.constant(0.0002)
numEpochs = 1000
batchSize = 128
displayStep = 100
#list of nodes  excpet x and y

(numExamples,xDim) = X_Mag.shape
(numExamples,yDim) = S_Mag.shape
windowSize = 20 ## window size required 

X = tf.placeholder(tf.float32,[None,windowSize,xDim],name = "X")
Y  =  tf.placeholder(tf.float32,[None,yDim], name = "Y")
keepProb = tf.placeholder(tf.float32)


## collecting weights in a dictionary

#filterHeight, filterWidth, inputDepth, numberOfFilters
AllfilterDims = [(4, 4, 1, 16),(2, 2, 16, 32)]

## (to apply or not, height,width) ## reduces by the stride in height and width
MaxPoolLayers = [(True,2,2),(True,2,2)] ##to be appplied after the first filter



### fully connected definitions
#lastchannel into the output layer reduced both in height and width
AllFullyConnectedDims = [32*129*5,800] ##does not include ydim


def getFinalFullyConDims(inputDims,AllfilterDims,MaxPoolLayer):
   
  
  pass


def filterWeigths(filterHeight, filterWidth, inputDepth, numberOfFilters):
  ## easy function for initialising and defining wirghts
  initialiser = tf.contrib.layers.variance_scaling_initializer()
  
  filterWeightTensor = tf.Variable(initialiser([filterHeight,filterWidth,inputDepth,numberOfFilters],dtype=tf.float32))
  return filterWeightTensor

def filterBisases(numberOfFilters):
  initialiser = tf.contrib.layers.variance_scaling_initializer()
  filterBiasTensor = tf.Variable(initialiser([numberOfFilters],dtype=tf.float32)) 
  return filterBiasTensor


def defineWeightsAndBias(AllfilterDims,AllFullyConnectedDims,yDim):
  
  ## he initialiisation
  initialiser = tf.contrib.layers.variance_scaling_initializer()
  
  
  
  ## filter weights and biases
  FilterWeights = [filterWeigths(filterHeight, filterWidth, inputDepth, numberOfFilters) \
             for (filterHeight, filterWidth, inputDepth, numberOfFilters) in AllfilterDims]
  
  
  Filterbiases = [filterBisases(numberOfFilters) for (filterHeight, filterWidth, inputDepth, numberOfFilters) in AllfilterDims]

  
  
  ##fully con weights and biasess
  
  FullyConWeights = [tf.Variable(initialiser([j,k],dtype=tf.float32)) \
                      for (j,k) in zip(AllFullyConnectedDims,AllFullyConnectedDims[1:]+[yDim])]
  
  
  FullyConBiases = [tf.Variable(initialiser([j],dtype=tf.float32))
                      for j in AllFullyConnectedDims[1:]+[yDim]]
  
  
  
  return FilterWeights, Filterbiases,FullyConWeights,FullyConBiases ##contains y incluede weights



def convolve2d(inputToLayer, singleFilterWeight, singleFilterBias, strides=1):
    #conv2 easy func
    convOutput = tf.nn.conv2d(inputToLayer, singleFilterWeight, strides=[1, strides, strides, 1], padding='SAME')
    convOutput = tf.nn.bias_add(convOutput, singleFilterBias)
    return tf.nn.relu(convOutput)


def maxpool2d(inputToLayer, filterSizeHeight,filterSizeWidth):
#     maxpool2d easy func
    return tf.nn.max_pool(inputToLayer, ksize=[1, filterSizeHeight, filterSizeWidth, 1], strides=[1, filterSizeHeight, filterSizeWidth, 1],
                          padding='SAME')





In [0]:
## accuracy metirc 
def RSquared(yAct, yPred):

    residual = tf.reduce_sum(tf.square(tf.subtract(yAct, yPred)))
    total = tf.reduce_sum(tf.square(tf.subtract(yAct, tf.reduce_mean(yAct))))
    r2 = tf.subtract(1.0, tf.math.divide(residual, total))
    return r2
  
  
## convert out put to time domian
def getOutputSound(xRawInp,xMag, YPred):
    S_Hat = np.multiply(xRawInp/xMag,YPred)
    S_Hat = librosa.istft(S_Hat.T, hop_length=512)

        
    return S_Hat
  
## signal ton noise ration
def getSNR(yRawInpTimeDomain,yPredReadyToWrite):
    minLength = min(len(yRawInpTimeDomain),len(yPredReadyToWrite))
    clippedY_True = yRawInpTimeDomain[:minLength]
    clippedY_Pred = yPredReadyToWrite[:minLength]
    signal  = np.dot(clippedY_True,clippedY_True.T)
    subtract = clippedY_True - clippedY_Pred
    noise = np.dot(subtract,subtract.T)
    
    return 10*math.log10(signal/noise)

In [0]:
def getNetworkOutput(xInp,FilterWeights, Filterbiases,FullyConWeights,FullyConBiases,MaxPoolLayer,keepProb):
    ##-1 to infer batch size, Height, Width, Depth
      
    xInp = tf.reshape(xInp, shape=[-1, 20, 513, 1])
    totalLayers = len(FilterWeights)+len(FullyConWeights)+ 1 ##for y 
    
    ## declarer empty list to save all outputs
    
    filterOutputList = []
    FullConOutputList = []

    ## drive through filters
       
    
    for layerPosition,(filWeight,filBias,(boolMaxpool,maxPoolHeight,maxPoolwidth)) in enumerate(zip(FilterWeights,Filterbiases,MaxPoolLayers)):
      
#       tf.print(layerPosition)
      if layerPosition == 0:
        
        output = convolve2d(xInp, filWeight, filBias)
        
        filterOutputList.append(output)
        
      else:
        
        output = convolve2d(output, filWeight, filBias)
        
        filterOutputList.append(output) ##collecting filter outputs
      
      if boolMaxpool == True:
        
        output = maxpool2d(output,maxPoolHeight,maxPoolwidth)
        
        
      filterOutputList.append(output) ##collecting max pool outputs
      

      maxPoolReshaped = tf.reshape(output, [-1, FullyConWeights[0].get_shape().as_list()[0]])
      



  ###drive through fully connected
    
    
    lastLayerPosition = len(FullyConWeights)-1
    for layerPosition,(weight,bias) in enumerate(zip(FullyConWeights,FullyConBiases)):

      if layerPosition == 0:
      
        fullConOutput = tf.nn.relu(tf.add(tf.matmul(maxPoolReshaped, weight), bias))
        FullConOutputList.append(fullConOutput)


      elif layerPosition <= lastLayerPosition-1:
      
        fullConOutput = tf.nn.relu(tf.add(tf.matmul(fullConOutput, weight), bias))
        FullConOutputList.append(fullConOutput)  
      
      else:
        
        fullConOutput = tf.add(tf.matmul(fullConOutput, weight), bias)
        fullConOutput = tf.nn.relu(fullConOutput,name='activation_relu')
        FullConOutputList.append(fullConOutput)
      
      
      ## dropout layer
      fullConOutput = tf.nn.dropout(fullConOutput, keepProb)
    
    return fullConOutput,filterOutputList,FullConOutputList




In [0]:
## required training
FilterWeights, Filterbiases,FullyConWeights,FullyConBiases = defineWeightsAndBias(AllfilterDims,AllFullyConnectedDims,yDim)

FinalOutput ,filterOutputList,FullConOutputList = getNetworkOutput(X,FilterWeights, Filterbiases,
                                                                   FullyConWeights,FullyConBiases,
                                                                   MaxPoolLayers,keepProb)

lossCalcu  = tf.losses.mean_squared_error(labels = Y ,predictions = FinalOutput)

AdamOptimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

train = AdamOptimizer.minimize(lossCalcu)
accuracy = RSquared(FinalOutput,Y)

initialise = tf.global_variables_initializer()

In [0]:
### training and running the sessions

with tf.Session() as currSess:


    x_train = gen2DImageFromAudio(X_Mag,20)
    

    y_train = S_Mag[19:,:]
    
    randomMatrix_Pred = np.random.rand(19,513)/10
    
    
    
    ##training 
    currSess.run(initialise)
    
    
    for epoch in range(997):
      for index, offset in enumerate(range(0, len(x_train), batchSize)):
        
        xBatch, yBatch = x_train[offset: offset + batchSize], y_train[offset: offset + batchSize]

        ## drop out train time keep Prob = 0.9
        currSess.run(train,feed_dict ={X:xBatch,Y:yBatch,keepProb: 0.9})
      
      
      ## drop out test time keep prob = 1
      loss = currSess.run([lossCalcu], feed_dict={X: x_train, Y: y_train,keepProb: 1})  
      accu = currSess.run([accuracy], feed_dict={X: x_train, Y: y_train,keepProb: 1})  
#       accu = 0
      
      print("Step " + str(epoch) + ", Loss= " + str(loss) + " Accuracy = "+ str(accu))
      
#     ###predictions for test sets

      trainPreds = currSess.run([FinalOutput], feed_dict={X: x_train,keepProb: 1})  
      paddedX_Train = np.concatenate((randomMatrix_Pred,trainPreds[0]), axis=0)
      s_Hat_Train = getOutputSound(X_Train.T,X_Mag,paddedX_Train)
      print(getSNR(sn_train_y,s_Hat_Train))



#     ##x1
    x_test_Mag = X1_Test_Mag
    x_test_Mag2D = gen2DImageFromAudio(X1_Test_Mag,20)
    
    testPreds1 = currSess.run([FinalOutput], feed_dict={X: x_test_Mag2D,keepProb: 1})  
    s_Hat1 = getOutputSound(X1_test[19:,:],x_test_Mag[19:,:],testPreds1[0])
  
    
    ##x2
    x_test_Mag = X2_Test_Mag
    x_test_Mag2D = gen2DImageFromAudio(X2_Test_Mag,20)

    testPreds2 = currSess.run([FinalOutput], feed_dict={X: x_test_Mag2D,keepProb: 1})  
    s_Hat2 = getOutputSound(X2_test[19:,:],x_test_Mag[19:,:],testPreds2[0])
    
    
    
    
    
    
   
    
    
    
    


Step 0, Loss= [0.09424651] Accuracy = [-7.7552357]
0.40876504123165097
Step 1, Loss= [0.085019074] Accuracy = [-9.412022]
0.8120432125968862
Step 2, Loss= [0.079244286] Accuracy = [-6.0099277]
1.1594083617523174
Step 3, Loss= [0.074644364] Accuracy = [-4.598857]
1.4403499357867764
Step 4, Loss= [0.0693533] Accuracy = [-2.9876082]
1.8185792172258195
Step 5, Loss= [0.06538179] Accuracy = [-2.360894]
2.0946579773448954
Step 6, Loss= [0.062185414] Accuracy = [-1.7160838]
2.365651227858133
Step 7, Loss= [0.05930011] Accuracy = [-1.5950494]
2.552570480289344
Step 8, Loss= [0.056057554] Accuracy = [-0.8579558]
2.9198450922718444
Step 9, Loss= [0.054182477] Accuracy = [-0.6044576]
3.1380306668303475
Step 10, Loss= [0.052432816] Accuracy = [-0.5553552]
3.254592261322272
Step 11, Loss= [0.05066496] Accuracy = [-0.51200056]
3.3631428618146133
Step 12, Loss= [0.04829106] Accuracy = [-0.3908962]
3.5619422794184885
Step 13, Loss= [0.049061943] Accuracy = [-0.57821083]
3.428062449868821
Step 14, Loss

In [0]:
### write to required files

librosa.output.write_wav('test_s_01_recons.wav', s_Hat1, sr = 16000)
librosa.output.write_wav('test_s_02_recons.wav', s_Hat2, sr = 16000)
librosa.output.write_wav('train_cleaned.wav', s_Hat_Train, sr = 16000)
